In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from mongo import Mongo
import plotly.express as px

price_repo = Mongo(db='trading_bot', collection='stock_price')
month_revenue_repo = Mongo(db='trading_bot', collection='month_revenue')
balance_sheet_repo = Mongo(db='trading_bot', collection='balance_sheet')
income_sheet_repo = Mongo(db='trading_bot', collection='income_sheet')
cash_flow_repo = Mongo(db='trading_bot', collection='cash_flow')

price_data = price_repo.get_data_by_stock_id('2330')
# month_revenue_data = month_revenue_repo.get_data_by_stock_id('1101')
# income_sheet = income_sheet_repo.get_data_by_stock_id('1101')
# balance_sheet = balance_sheet_repo.get_data_by_stock_id('1101')
# cash_flow = cash_flow_repo.get_data_by_stock_id('1101')
# all_price_data = price_repo.get_all_data()
# all_month_revenue_data = month_revenue_repo.get_all_data()
# all_income_sheet = income_sheet_repo.get_all_data()
# all_balance_sheet = balance_sheet_repo.get_all_data()
# all_cash_flow = cash_flow_repo.get_all_data()

In [6]:
price_data

""
